In [1]:
import os
import cv2
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

# Constants
IMAGE_SIZE = (224, 224)
SUPPORTED_FORMATS = ['.jpg', '.jpeg', '.png']

In [2]:
class ImagePreprocessor:
    """Skin Cancer Image preprocessing pipeline"""
    
    @staticmethod
    def hair_remove(image):
        """Remove hair from skin images"""
        try:
            grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
            blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
            _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
            final_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)
            return final_image
        except Exception as e:
            print(f"Error in hair removal: {str(e)}")
            return image

    @staticmethod
    def sharpen_image(image):
        """Sharpen image using unsharp masking"""
        gaussian = cv2.GaussianBlur(image, (0, 0), 2.0)
        return cv2.addWeighted(image, 1.5, gaussian, -0.5, 0)

In [3]:
def preprocess_image(image, target_size=(224, 224)):
    """Apply all preprocessing steps to an image"""
    preprocessor = ImagePreprocessor()
    
    image = preprocessor.hair_remove(image)
    image = preprocessor.sharpen_image(image)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST)
    
    return image

In [4]:
def create_binary_mask(image):
    """Create binary mask from image using binary thresholding"""
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Apply gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply thresholding
    _, binary = cv2.threshold(blurred, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Invert the mask (1 for lesion, 0 for background)
    mask = 1 - binary
    
    # Clean up mask with morphological operations
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    return mask

In [5]:
def process_and_organize_dataset(source_path, destination_path):
    """Process images and organize them into the new structure"""
    source_path = Path(source_path)
    destination_path = Path(destination_path)
    
    # Process each split (train, test, val)
    splits = ['train_directory', 'test_directory', 'validation_directory']
    for split in splits:
        split_path = source_path / split
        
        dest_split = split
        
        for category in ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']:
            category_path = split_path / category
            if not category_path.exists():
                continue
                
            print(f"Processing {split}/{category}...")
            
            # Process each image in the category
            for img_file in tqdm([f for ext in SUPPORTED_FORMATS for f in category_path.glob(f'*{ext}')]):
                img = cv2.imread(str(img_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Apply preprocessing
                processed_img = preprocess_image(img.copy(), target_size=(224, 224))
                
                # Generate filename without extension
                filename = img_file.stem
                
                # Create and save binary thresholding ground truth mask from preprocessed image
                mask = create_binary_mask(processed_img)
                mask_save = (mask * 255).astype(np.uint8)
                mask_path = destination_path / 'ground_truth' / dest_split / category / f"{filename}.jpg"
                cv2.imwrite(str(mask_path), mask_save)
                
                # Create and save binary thresholding segmented image
                segmented = processed_img.copy()
                segmented[mask == 0] = 0
                segmented_path = destination_path / 'segmented_bt' / dest_split / category / f"{filename}.jpg"
                cv2.imwrite(str(segmented_path), cv2.cvtColor(segmented, cv2.COLOR_RGB2BGR))

In [7]:
base_directory = '/kaggle/working/segmented_bt_images'
os.mkdir(base_directory)

subfolders = ['ground_truth', 'segmented_bt']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    path = os.path.join(base_directory, subf)
    os.mkdir(path)
    for dirc in directory:
        path = os.path.join(base_directory, subf, dirc)
        os.mkdir(path)
        for cls in classes:
            path = os.path.join(base_directory, subf, dirc, cls)
            os.mkdir(path)

In [8]:
source_path = "/kaggle/input/multiclassskincancer"
destination_path = "/kaggle/working/segmented_bt_images"
process_and_organize_dataset(source_path, destination_path)

Processing train_directory/nv...


100%|██████████| 5115/5115 [05:07<00:00, 16.62it/s]


Processing train_directory/mel...


100%|██████████| 5950/5950 [02:53<00:00, 34.21it/s]


Processing train_directory/bkl...


...

100%|██████████| 24/24 [00:01<00:00, 12.83it/s]


Processing validation_directory/vasc...


100%|██████████| 10/10 [00:00<00:00, 14.83it/s]


Processing validation_directory/df...


100%|██████████| 6/6 [00:00<00:00, 12.44it/s]


In [9]:
base_dir = '/kaggle/working/segmented_bt_images'
subfolders = ['ground_truth', 'segmented_bt']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    print(subf)
    for dirc in directory:
        print(dirc)
        for cls in classes:
            path = os.path.join(base_dir, subf, dirc, cls)
            print(f"{cls}    : ", len(os.listdir(path)))

ground_truth
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
segmented_bt
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
